In [88]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


#  Packages & Imports

In [219]:
%load_ext autoreload
%aimport data_prep
%run data_prep.py
%aimport modeling
%run modeling.py
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load prepared Data

In [104]:
#Load Datasets
dataset = pd.read_csv("./FE_Final/train_data_prepared.csv")
train_data = dataset[dataset.is_test == 0]
target = pd.read_csv("./original_datasets/training_set_labels.csv")
target=target[["status_group"]]

train_data  = train_data.drop(['subvillage','date_recorded','wpt_name','recorded_by','is_test'],axis=1)

NameError: name 'pd' is not defined

In [105]:
train_data.head()

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,num_private,basin,region,...,source_type,source_class,waterpoint_type,waterpoint_type_group,density,age,Flag_age,Flag_date_recorded,days_since_recoreded,distance
0,69572,6000.0,other,1390,other,34.938093,-9.856322,0,Lake Nyasa,Iringa,...,spring,groundwater,communal standpipe,communal standpipe,0.000116,20.000000,0,0,"(2901.0, 2920.0]",1783.545310
1,8776,0.0,other,1399,other,34.698766,-2.147466,0,Lake Victoria,Mara,...,rainwater harvesting,surface,communal standpipe,communal standpipe,0.000161,9.000000,0,0,"(1923.999, 2196.0]",931.445791
2,34310,25.0,other,686,other,37.460664,-3.821329,0,Pangani,Manyara,...,dam,surface,communal standpipe multiple,communal standpipe,0.000175,10.000000,0,0,"(2196.0, 2221.0]",1126.306889
3,67743,0.0,Unicef,263,other,38.486161,-11.155298,0,Ruvuma / Southern Coast,Mtwara,...,borehole,groundwater,communal standpipe multiple,communal standpipe,0.000046,33.000000,0,0,"(2221.0, 2241.0]",1949.338824
4,19728,0.0,other,0,other,31.130847,-1.825359,0,Lake Victoria,Kagera,...,rainwater harvesting,surface,communal standpipe,communal standpipe,0.000000,14.389401,1,0,"(2790.0, 2901.0]",1025.769078


In [106]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 42 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
funder                   59400 non-null object
gps_height               59400 non-null int64
installer                59400 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
num_private              59400 non-null int64
basin                    59400 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
scheme_management        59400 non-null object
scheme_name              31234 non-null object
permit                   56344 non-null object
c

In [122]:
train_data_trees = train_data.copy()
train_data_lr = train_data.copy()

train_label_encode = label_encoder(train_data_trees)
train_onehot_encode = onehot_encode(train_data_lr)



# of features before: 42
# of features after: 4999


## Baseline Model (Random Forest)

In [108]:
X_train, X_test, y_train, y_test = train_test_split(train_label_encode, target, train_size=0.8)

rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

rf.fit(X_train, y_train)

rf_score = rf.score(X_test,y_test)

print('Accuracy:', rf_score)


Accuracy: 0.8121212121212121


## Baseline Model (XGBoost)

In [109]:
xgb_model = XGBClassifier(objective="multi:softprob")

xgb_model.fit(X_train, y_train)

xgb_score = xgb_model.score(X_test,y_test)

print('Accuracy:', xgb_score)


Accuracy: 0.7455387205387205


# Multi-class logReg

In [113]:
train_onehot_encode.head()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,...,extraction_type_group_wind-powered,scheme_management_Company,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority,scheme_management_other
0,69572,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999,...,0,0,0,0,1,0,0,0,0,0
1,8776,0.0,1399,34.698766,-2.147466,0,20,2,280,2010,...,0,0,0,0,0,0,0,0,0,1
2,34310,25.0,686,37.460664,-3.821329,0,0,4,250,2009,...,0,0,0,0,1,0,0,0,0,0
3,67743,0.0,263,38.486161,-11.155298,0,0,63,58,1986,...,0,0,0,0,1,0,0,0,0,0
4,19728,0.0,0,31.130847,-1.825359,0,18,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [114]:
X_train, X_test, y_train, y_test = train_test_split(train_onehot_encode, target, train_size=0.8)

clf = LogisticRegression(multi_class="ovr").fit(train_onehot_encode, target)
clf_Score = clf.score(X_test,y_test)
print('Accuracy:', clf_Score)

Accuracy: 0.7207912457912458


## KNN

##### Only latitude and longitude and height were used in order to try and see if there is a relationship  between the location of the well and the target 

In [128]:
X_train, X_test, y_train, y_test = train_test_split(train_label_encode[['latitude','longitude']], target, train_size=0.8)

In [130]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(algorithm='auto', leaf_size=5, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=1,
           weights='uniform')

neighbors =  knn.fit(X_train, y_train)
neighbors.score(X_test,y_test)

0.680050505050505

### Full Grid Search for Random Forest

##### The best parameters will be used to output a result for the different combination of feature creation and imputing 

In [228]:
data = pd.read_csv("./original_datasets/initial_joined.csv")

In [211]:
dataset = data
to_int = {'functional':1,'non functional':2,'functional needs repair':3}
to_cat = {1:'functional',2:'non functional',3:'functional needs repair'}

y_Train = dataset['status_group'].loc[(dataset['is_test'].isin([0]) )]
y   = y_Train.replace(to_int).copy()
dataset = dataset.drop('status_group',axis=1).copy()
   
    ### Label encode
dataset = label_encoder(dataset).copy()
    ### Divide test and train 
X_Train= dataset.loc[(dataset['is_test'].isin([0]) )].drop('is_test',axis=1)
test = dataset.loc[(dataset['is_test'].isin([1]) )].drop(['is_test'],axis=1)

    #### Set ID aside for test 
ID = test['id'] 

    ### If train_id set to Trues the the ID will be kept as feature

X = X_Train.set_index('id')
test = test.set_index('id')

In [ ]:

RF_parameters = {'n_estimators': [100,200 ],
                                             'bootstrap': [True,False],
                                             'max_depth': [80, 100 ,120],
                                             'max_features': ['auto',16 ,32],
                                             'min_samples_leaf': [ 2,  5 ],
                                             'min_samples_split': [ 5,  8 ],
                                            'random_state':[random_seed],
                                                'n_jobs':[-1]}
rf = GridSearchCV(RandomForestClassifier(),
                                 param_grid= RF_parameters,
                                 cv=KFold(3,random_state = random_seed),verbose = True,n_jobs=-1)



rf.fit(X, y)

                    

In [215]:
rf.best_estimator_                                                           

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=6666, verbose=0,
            warm_start=False)

In [47]:
feature_importances = pd.DataFrame(rf.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

### Full Grid Search on XGBoost 

##### The following parameters will be defined to any XGBoost used later

In [ ]:
param_grid = {'learning_rate': [0.01, 0.1], 
          'max_depth': [4,8,12],
          'min_child_weight': [3,5,10,20,35,50],
          'subsample': [0.5, 0.75],
          'colsample_bytree': [0.5, 0.75],
          'n_estimators': [100, 300],
              'random_state':[random_seed]
}

model = xgb.XGBClassifier()


to_int = {'functional':1,'non functional':2,'functional needs repair':3}
to_cat = {1:'functional',2:'non functional',3:'functional needs repair'}

y_Train = dataset['status_group'].loc[(dataset['is_test'].isin([0]) )]
y_Train   = y_Train.replace(to_int).copy()
dataset = dataset.drop('status_group',axis=1).copy()
   
    ### Label encode
dataset = label_encoder(dataset).copy()
    ### Divide test and train 
X_Train= dataset.loc[(dataset['is_test'].isin([0]) )].drop('is_test',axis=1)
test = dataset.loc[(dataset['is_test'].isin([1]) )].drop(['is_test'],axis=1)

    #### Set ID aside for test 
ID = test['id'] 

    ### If train_id set to Trues the the ID will be kept as feature

X_Train = X_Train.set_index('id')
test = test.set_index('id')

xg = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = KFold(5,random_state=random_seed),
                    n_jobs = -1,
                    scoring = 'accuracy',
                    verbose=True)
xg.fit(X_Train,y_Train)
print(xg.cv_results_)
print(xg.best_params_)
print(xg.best_score_ )

XGBClassifier(subsample= 0.75, random_state=random_seed,
         n_estimators =  300, min_child_weight= 20, 
         max_depth=90, learning_rate= 0.1, colsample_bytree= 0.75,  n_jobs=-1, verbose = True)

In [128]:
 Xll = data[['longitude','latitude']]
yll = target

In [73]:
knn_parameters = {'n_neighbors':[1, 5,10,15 ],'weights':['uniform'], 'algorithm':['auto'],'leaf_size':[5 ,10, 20],
                  'p':[1,2,3],'metric':['minkowski']}
knn = GridSearchCV(KNeighborsClassifier(),verbose =True,
                                 param_grid=knn_parameters,
                                 cv=KFold(5,random_state = 42),return_train_score=True)
X_train, X_test, y_train, y_test = train_test_split(
    Xll, yll, random_state=42)
knn.fit(Xll, yll)
knn.cv_results_

In [131]:
knn.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=5, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=1,
           weights='uniform')

### <p style="color:darkred"> Scores for Random Forest Tried on diffrent combinations of features and imputing, Cross validation Scores are printed and the prediction directly saved to a file really for submission </P>

#### Dataset : 'Initial'

In [220]:
data = pd.read_csv("./original_datasets/initial_joined.csv")
scores , preds = test_score(data ,'initial')

The average of the cross validation with Random Forest:0.8148653198653198


#### Dataset :'imputed'

In [221]:
data1 = pd.read_csv("./FE_steps/1 train_data_imputed.csv")
scores1 , preds1 = test_score(data1 ,'1 train_data_imputed')

The average of the cross validation with Random Forest:0.8153198653198654


#### Dataset :'impute_age_days_recorded_density_PTR'

In [222]:
data2 = pd.read_csv("./FE_steps/2 train_data_impute_age_dayse_recorded_density.csv")
scores2 , preds2 = test_score(data2 ,'2 train_data_impute_age_days_recorded_density_PTR')

The average of the cross validation with Random Forest:0.8143602693602695


#### Dataset :'NOIMPUTE_age_days_recorded_density_PTR'

In [223]:
data3 = pd.read_csv("./FE_steps/3 train_data_NOIMPUTE_age_dayse_recorded_density.csv")
scores3 , preds3 = test_score(data3 ,'3 train_data_NOIMPUTE_age_days_recorded_density_PTR')

The average of the cross validation with Random Forest:0.8144612794612796


#### Dataset :'NOIMPUTE_age_days_recorded_density_capital_PTR'

In [224]:
data4 = pd.read_csv("./FE_steps/4 train_data_NOIMPUTE_age_dayse_recorded_density_capital.csv")
scores4 , preds4 = test_score(data4 ,'4 train_data_NOIMPUTE_age_days_recorded_density_capital_PTR')

The average of the cross validation with Random Forest:0.815959595959596


#### Dataset :'NOIMPUTE_age_dayse_recorded_density_capital_SHORTLISTING.'

In [225]:
data5 = pd.read_csv("./FE_steps/5 train_data_NOIMPUTE_age_dayse_recorded_density_capital_SHORTLISTING.csv")
scores5 , preds5 = test_score(data5 ,'5 train_data_NOIMPUTE_age_dayse_recorded_density_capital_SHORTLISTING_PTR')

The average of the cross validation with Random Forest:0.8154040404040404


#### Dataset :'gps_height_ONLY_imputed'

In [226]:
data6 = pd.read_csv("./FE_steps/6 train_data_gps_height_ONLY_imputed.csv")
scores6 , preds6 = test_score(data6 ,'6 train_data_gps_height_ONLY_imputed')

The average of the cross validation with Random Forest:0.8154545454545454


### Dataset :'capital_ONLY_latitude_longitude_PTR' <p style="color:red">This is our best score which is used for submission</p>

In [227]:
data7 = pd.read_csv("./FE_steps/7 train_data_capital_ONLY.csv")
scores7 , preds7 = test_score(data7 ,'7 train_data_capital_ONLY')

The average of the cross validation with Random Forest:0.8162962962962963


#### As seen here all of them have an acceptable score due to the parameters used from the best estimator of grid search , after testing  the highest score was with prediction 7 which includes the imputings ogf latitude and longitude, PTR (pupil per teacher) and Distance to capital.
#### Now in order to increase the score  7 predictions from each of the above datasets will be done on the train set in a kfold manner, predicting on the left out for each of the folds.
#### These 7 new predictions will now become features for a new model. This new model will train on the 7 predictions of the initial train set and will predict from the 7 predictions made on the test.

In [108]:
datasets = { 'data': data , 'data1' : data1 , 'data2':data2 , 'data3':data3 , 'data4': data4 , 'data5':data5
            , 'data6':data6 , 'data7':data7}

In [ ]:
model =    [ RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=32, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=random_seed, verbose=1, warm_start=False)]
S_train ={}
S_test = {}
for i in datasets:
    dataset = datasets[i]
    to_int = {'functional':1,'non functional':2,'functional needs repair':3}
    to_cat = {1:'functional',2:'non functional',3:'functional needs repair'}

    y_Train = dataset['status_group'].loc[(dataset['is_test'].isin([0]) )]
    y_Train   = y_Train.replace(to_int).copy()
    dataset = dataset.drop('status_group',axis=1).copy()
   
    ### Label encode
    dataset = label_encoder(dataset).copy()
    ### Divide test and train 
    X_Train= dataset.loc[(dataset['is_test'].isin([0]) )].drop('is_test',axis=1)
    test = dataset.loc[(dataset['is_test'].isin([1]) )].drop(['is_test'],axis=1)

    #### Set ID aside for test 
    ID = test['id'] 

    ### If train_id set to Trues the the ID will be kept as feature

    X_Train = X_Train.set_index('id')
    test = test.set_index('id')
    
    S_train[i], S_test[i] = stacking(model,                     # list of models
                           X_Train, y_Train, test,   # data
                           regression=False,           # classification task (if you need 
                                                       #     regression - set to True)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and vote
                           needs_proba=False,          # predict class labels (if you need 
                                                       #     probabilities - set to True) 
                           save_dir=None,              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=accuracy_score,      # metric: callable
                           n_folds=5,                  # number of folds
                           stratified=False,            # stratified split for folds
                           shuffle=False,               # shuffle the data
                           random_state=random_seed,
                                    verbose =2)                  # print all info
    

#### Stacking the Train and test predictions 

In [157]:
tpred=pd.DataFrame(columns= ['data', 'data1'  , 'data2' , 'data3' , 'data4', 'data5'
            , 'data6' , 'data7'])
testp=pd.DataFrame(columns= ['data', 'data1'  , 'data2' , 'data3' , 'data4', 'data5'
            , 'data6' , 'data7'])
for i in S_train :
    tpred[i]=[int(x) for x in S_train[i]]
for i in S_test :
    testp[i]=[int(x) for x in S_test[i]]

#### XGBoost classifier with paramerters obtained from a previous GridSearch trained on the prediction of the 8 random forests form each of the datasets.

In [158]:
xgbs = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=12, min_child_weight=3, missing=None, n_estimators=150,
       n_jobs=1, nthread=None, objective='multi:softprob',
       random_state=random_seed, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.5, verbose=True)  
xgbs.fit(tpred, y_Train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=12, min_child_weight=3, missing=None, n_estimators=150,
       n_jobs=1, nthread=None, objective='multi:softprob',
       random_state=6666, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.5, verbose=True)

#### Predicting from the stacked test predicitons

In [161]:
testpredictionsall = xgbs.predict(testp)

In [163]:

data = {'ID': ID, 'status_group': testpredictionsall}
submit = pd.DataFrame(data=data)
submit['status_group'] = submit.status_group.replace(to_cat)
submit.to_csv('predictions/allstack8.csv', index=False)

## <p style="color:darkred"> This yielded a score of 0.8185 on the compitetion , which is worst then the previous scores , so this techinique did not help </p>

##### The above stacking did not help improve our score , a different way of stacking will now be used.On the train dataset 5 , which gave on the highest score , an XGBClassifier and RandomForesct will train and predict on the train set and as earlier use the cross validation folds to fill the train prediction. These two prediction will be stacked and another XGBClassifier will be trained and will predict on the test set.

In [60]:
models = [
    
    XGBClassifier(subsample= 0.75, random_state=random_seed,
         n_estimators =  300, min_child_weight= 20, 
         max_depth=90, learning_rate= 0.1, colsample_bytree= 0.75,  n_jobs=-1, verbose = True),
        
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=32, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=random_seed, verbose=1, warm_start=False)

]

In [61]:
to_int = {'functional':1,'non functional':2,'functional needs repair':3}
to_cat = {1:'functional',2:'non functional',3:'functional needs repair'}

y_Train = data5['status_group'].loc[(data5['is_test'].isin([0]) )]
y_Train   = y_Train.replace(to_int).copy()
dataset = data5.drop('status_group',axis=1).copy()
   
    ### Label encode
data5 = label_encoder(data5).copy()
    ### Divide test and train 
X_Train= data5.loc[(data5['is_test'].isin([0]) )].drop('is_test',axis=1)
test = data5.loc[(data5['is_test'].isin([1]) )].drop(['is_test'],axis=1)

    #### Set ID aside for test 
ID = test['id'] 

    ### If train_id set to Trues the the ID will be kept as feature

X_Train = X_Train.set_index('id')
test = test.set_index('id')

In [62]:
St_train, St_test = stacking(models,                     # list of models
                           X_Train, y_Train, test,   # data
                           regression=False,           # classification task (if you need 
                                                       #     regression - set to True)
                           mode='oof_pred_bag',        # mode: oof for train set, predict test 
                                                       #     set in each fold and vote
                           needs_proba=False,          # predict class labels (if you need 
                                                       #     probabilities - set to True) 
                           save_dir=None,              # do not save result and log (to save 
                                                       #     in current dir - set to '.')
                           metric=accuracy_score,      # metric: callable
                           n_folds=5,                  # number of folds
                           stratified=False,            # stratified split for folds
                           shuffle=True,               # shuffle the data
                           random_state=random_seed,             # ensure reproducibility
                           verbose=2)                  # print all info

task:         [classification]
n_classes:    [3]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [XGBClassifier]
    fold  0:  [0.81111111]
    fold  1:  [0.80513468]
    fold  2:  [0.80530303]
    fold  3:  [0.81380471]
    fold  4:  [0.81304714]
    ----
    MEAN:     [0.80968013] + [0.00374745]
    FULL:     [0.80968013]

model  1:     [RandomForestClassifier]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   53.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    fold  0:  [0.81161616]


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   53.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    fold  1:  [0.80993266]


...

    fold  2:  [0.80690236]


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   53.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    fold  3:  [0.81489899]


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   53.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished


    fold  4:  [0.81666667]
    ----
    MEAN:     [0.81200337] + [0.00348106]
    FULL:     [0.81200337]



In [45]:
xgbst = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=12, min_child_weight=3, missing=None, n_estimators=150,
       n_jobs=1, nthread=None, objective='multi:softprob',
       random_state=random_seed, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.5, verbose=True)  
NX_Train = pd.DataFrame(St_train)

scores= cross_val_score(xbst, 
                NX_Train, y_Train,scoring='accuracy', cv=KFold(5,random_state= random_seed))

In [63]:
NX_Train = pd.DataFrame(S_train)
test = pd.DataFrame(S_test)

In [64]:
xgbst.fit(NX_Train, y_Train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=12, min_child_weight=3, missing=None, n_estimators=150,
       n_jobs=1, nthread=None, objective='multi:softprob',
       random_state=6666, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.5, verbose=True)

### Score of 2nd stacking

In [55]:
xgbst.best_score_

0.812003367003367

In [ ]:
testpt = xgbs.predict(test)

In [47]:
data = {'ID': ID, 'status_group': testp]}
submit = pd.DataFrame(data=data)
submit['status_group'] = submit.status_group.replace(to_cat)
submit.to_csv('predictions/stack2.csv', index=False)

##### This stacking as well still did not help to improve the predictions on the pump it up challenge , it seemed more relevant to take the random forest trained on test set 5 since it had the highest accuracy and many of the features created were relevant in it.


### The final score on the sumission on the website is 82.40